In [1]:
import struct
import copy
import json
import os
import shutil
import time

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# CUSTOM
from utils import *
from baxter_pickplace_interface import *
from pydpp.dpp import DPP

# ROS
import rospy
import gazebo_msgs.srv
import geometry_msgs.msg
import std_msgs.msg
import tf.transformations

# BAXTER
import baxter_core_msgs.srv
import baxter_interface

# GQCNN
from autolab_core import YamlConfig, Logger
import autolab_core.utils as utils
from gqcnn import get_gqcnn_model, get_gqcnn_trainer, utils as gqcnn_utils
from perception import (BinaryImage, CameraIntrinsics, ColorImage, DepthImage,
                        RgbdImage)
from visualization import Visualizer2D as vis
from gqcnn.grasping import GraspAction
from gqcnn.grasping import (RobustGraspingPolicy,
                            CrossEntropyRobustGraspingPolicy, RgbdImageState,
                            FullyConvolutionalGraspingPolicyParallelJaw,
                            FullyConvolutionalGraspingPolicySuction)
from gqcnn.utils import GripperMode
from gqcnn import NoValidGraspsException

# FOR VISUALIZATION
from matplotlib import pyplot as plt
%matplotlib inline

/home/dof6/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
W0831 03:17:44.566159 139934685038336 deprecation_wrapper.py:119] From /home/dof6/.local/lib/python2.7/site-packages/gqcnn/utils/enums.py:44: The name tf.RunOptions is deprecated. Please use tf.compat.v1.RunOptions instead.

I0831 03:17:44.574744 139934685038336 driver.py:120] Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt


root       INFO     Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt


I0831 03:17:44.594222 139934685038336 driver.py:120] Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


root       INFO     Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


W0831 03:17:45.027491 139934685038336 kinect2_sensor.py:15] Unable to import pylibfreenect2. Python-only Kinect driver may not work properly.


...

root       WARNING  Failed to import ROS in phoxi_sensor.py. PhoXiSensor functionality unavailable.


I0831 03:17:45.090197 139934685038336 acceleratesupport.py:13] OpenGL_accelerate module loaded


OpenGL.acceleratesupport INFO     OpenGL_accelerate module loaded


I0831 03:17:45.094729 139934685038336 arraydatatype.py:270] Using accelerated ArrayDatatype


OpenGL.arrays.arraydatatype INFO     Using accelerated ArrayDatatype


In [2]:
# Image Cropping Info
cx = 285
cy = 255
wx = 115
wy = 135
crop_size = int(96/2)
target_dir = './dynamic_grasp_data/'
fname = ['tf_depth_ims_', 'grasps_', 'grasp_metrics_']
hand_pose_dim = 6

def load_policy(iteration, wx = 115, wy = 135, method = ''):
    if iteration==0:
        model_name = 'GQCNN-4.0-PJ'
    else:
        model_name = 'GQCNN-4.0-PJ-'+str(iteration)+'-'+method+"-GQ"
    config_filename = None
    fully_conv = False

    __file__ = os.path.abspath('')

    depth_im_filename = None
    segmask_filename = None
    camera_intr_filename = None
    if iteration == 0:
        model_dir = None
    else:
        model_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),"scripts/grasp_learner/models/")

    assert not (fully_conv and depth_im_filename is not None
                and segmask_filename is None
                ), "Fully-Convolutional policy expects a segmask."

    if camera_intr_filename is None:
        camera_intr_filename = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                            "scripts/grasp_experts/data/calib/primesense/primesense.intr")

    # Set model if provided.
    if model_dir is None:
        model_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                 "scripts/grasp_experts/models")
    model_path = os.path.join(model_dir, model_name)

    # Get configs.
    model_config = json.load(open(os.path.join(model_path, "config.json"), "r"))
    try:
        gqcnn_config = model_config["gqcnn"]
        gripper_mode = gqcnn_config["gripper_mode"]
    except KeyError:
        gqcnn_config = model_config["gqcnn_config"]
        input_data_mode = gqcnn_config["input_data_mode"]
        if input_data_mode == "tf_image":
            gripper_mode = GripperMode.LEGACY_PARALLEL_JAW
        elif input_data_mode == "tf_image_suction":
            gripper_mode = GripperMode.LEGACY_SUCTION
        elif input_data_mode == "suction":
            gripper_mode = GripperMode.SUCTION
        elif input_data_mode == "multi_suction":
            gripper_mode = GripperMode.MULTI_SUCTION
        elif input_data_mode == "parallel_jaw":
            gripper_mode = GripperMode.PARALLEL_JAW
        else:
            raise ValueError(
                "Input data mode {} not supported!".format(input_data_mode))

    # Set config.
    if config_filename is None:
        if (gripper_mode == GripperMode.LEGACY_PARALLEL_JAW
                or gripper_mode == GripperMode.PARALLEL_JAW):
            if fully_conv:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/fc_gqcnn_pj.yaml")
            else:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/gqcnn_pj.yaml")
        elif (gripper_mode == GripperMode.LEGACY_SUCTION
              or gripper_mode == GripperMode.SUCTION):
            if fully_conv:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/fc_gqcnn_suction.yaml")
            else:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/gqcnn_suction.yaml")
    else:
        config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), config_filename)
    # Read config.
    config = YamlConfig(config_filename)
    inpaint_rescale_factor = config["inpaint_rescale_factor"]
    policy_config = config["policy"]

    # Make relative paths absolute.
    if "gqcnn_model" in policy_config["metric"]:
        policy_config["metric"]["gqcnn_model"] = model_path
        if not os.path.isabs(policy_config["metric"]["gqcnn_model"]):
            policy_config["metric"]["gqcnn_model"] = os.path.join(
                os.path.dirname(os.path.realpath(__file__)), "..",
                policy_config["metric"]["gqcnn_model"])

    # Setup sensor.
    camera_intr = CameraIntrinsics.load(camera_intr_filename)

    # Set input sizes for fully-convolutional policy.
    if fully_conv:
        policy_config["metric"]["fully_conv_gqcnn_config"][
            "im_height"] = wy*2
        policy_config["metric"]["fully_conv_gqcnn_config"][
            "im_width"] = wx*2

    # Init policy.
    if fully_conv:
        # TODO(vsatish): We should really be doing this in some factory policy.
        if policy_config["type"] == "fully_conv_suction":
            policy = FullyConvolutionalGraspingPolicySuction(policy_config)
        elif policy_config["type"] == "fully_conv_pj":
            policy = FullyConvolutionalGraspingPolicyParallelJaw(policy_config)
        else:
            raise ValueError(
                "Invalid fully-convolutional policy type: {}".format(
                    policy_config["type"]))
    else:
        policy_type = "cem"
        if "type" in policy_config:
            policy_type = policy_config["type"]
        if policy_type == "ranking":
            policy = RobustGraspingPolicy(policy_config)
        elif policy_type == "cem":
            policy = CrossEntropyRobustGraspingPolicy(policy_config)
        else:
            raise ValueError("Invalid policy type: {}".format(policy_type))
            
    number_of_samples = 20
    def best_action(policy, depth_data, color_data):
        # Read images.
        depth_im = DepthImage(depth_data, frame=camera_intr.frame)
        color_im = ColorImage(color_data, frame=camera_intr.frame)

        # Inpaint.
        depth_im = depth_im.inpaint(rescale_factor=inpaint_rescale_factor)
        segmask = depth_im.invalid_pixel_mask().inverse()
        
        # Create state.
        rgbd_im = RgbdImage.from_color_and_depth(color_im, depth_im)
        state = RgbdImageState(rgbd_im, camera_intr, segmask=segmask)
        
        # Query policy.
        policy_start = time.time()
        grasps, q_values = policy.action_set(state)
            
        idx = np.argmax(q_values)
        grasp = grasps[idx]
        q_value = q_values[idx]
        
        action = GraspAction(grasp,q_value)
        print("Grasp Quality : %f"%q_value)
        return action
    
    def sample_action(policy, depth_data, color_data):
        # Read images.
        depth_im = DepthImage(depth_data, frame=camera_intr.frame)
        color_im = ColorImage(color_data, frame=camera_intr.frame)

        # Inpaint.
        depth_im = depth_im.inpaint(rescale_factor=inpaint_rescale_factor)
        segmask = depth_im.invalid_pixel_mask().inverse()
        
        # Create state.
        rgbd_im = RgbdImage.from_color_and_depth(color_im, depth_im)
        state = RgbdImageState(rgbd_im, camera_intr, segmask=segmask)
        
        # Query policy.
        policy_start = time.time()
        grasps, q_values = policy.action_set(state)
            
        if method=='Greedy':
            idx = np.argmax(q_values)
            grasp = grasps[idx]
            q_value = q_values[idx]
        elif method=='Random':
            idx = np.random.choice(len(q_values),size=1)[0]
            grasp = grasps[idx]
            q_value = q_values[idx]
        elif method=='EXP3':
            p = np.exp((q_values - np.max(q_values))/0.05)
            p = p/np.sum(p)
            
            idx = np.random.choice(len(p),size=1,p=p)[0]
            grasp = grasps[idx]
            q_value = q_values[idx]

        action = GraspAction(grasp,q_value)
        print("Grasp Quality : %f"%q_value)
        return action
            
    return policy, best_action, sample_action

In [3]:
rospy.init_node("ik_pick_and_place_demo")

In [4]:
limb = "left"
hover_distance = 0.15
pnp = PickAndPlace(limb, hover_distance)
pnp.move_to_start()

[INFO] [1567189065.814576, 8.803000]: Robot Enabled


In [5]:
rospy.wait_for_service('/gazebo/spawn_sdf_model')
rospy.wait_for_service('/gazebo/spawn_urdf_model')
spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', gazebo_msgs.srv.SpawnModel)
spawn_urdf = rospy.ServiceProxy('/gazebo/spawn_urdf_model', gazebo_msgs.srv.SpawnModel)
delete_model = rospy.ServiceProxy('/gazebo/delete_model', gazebo_msgs.srv.DeleteModel)
set_model = rospy.ServiceProxy('/gazebo/set_model_state', gazebo_msgs.srv.SetModelState)

camera_quaternion = tf.transformations.quaternion_from_euler(0., np.pi / 2., 0.)
spawn_gazebo_model(spawn_urdf, "realsense", "rs200", 0.65, 0.3, 0.7825+0.7, 
                   qx=camera_quaternion[0],qy=camera_quaternion[1],qz=camera_quaternion[2],qw=camera_quaternion[3])
spawn_gazebo_model(spawn_sdf, "cafe_table", "cafe_table", 0.8, 0.3, 0.0)

file_list = os.listdir('/home/dof6/Downloads/kit/')
model_name_list = []
for file_name in file_list:
    name = file_name.replace(".obj","")
    trimesh_data = trimesh.load('/home/dof6/Downloads/kit/'+name+'.obj')
    if np.min(trimesh_data.bounds[1] - trimesh_data.bounds[0]) > 0.038 and np.max(trimesh_data.bounds[1] - trimesh_data.bounds[0]) > 0.10:
        model_name_list.append(name)
number_of_models = len(model_name_list)
print(number_of_models)

21


In [6]:
depth_cam_rotation_matrix = np.zeros([4,4])
depth_cam_rotation_matrix[1,0] = -1.
depth_cam_rotation_matrix[0,1] = -1.
depth_cam_rotation_matrix[2,2] = -1.
depth_cam_rotation_matrix[3,3] = 1.
depth_cam_rotation_matrix[0,3] = .65
depth_cam_rotation_matrix[1,3] = 0.3-.03
depth_cam_rotation_matrix[2,3] = 1.4825

In [7]:
method = 'EXP3'
policy, best_action, sample_action = load_policy(0, wx=wx, wy=wy, method=method)
num_trials = 10
model_grasp_result_list = np.zeros([number_of_models, num_trials, 2])

/home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_experts/models/GQCNN-4.0-PJ


In [8]:
rospy.sleep(2.)

for model_idx in range(number_of_models):
    name = model_name_list[model_idx]
    rnd_x = 0.025*(2.*np.random.uniform()-1.) + 0.6
    rnd_y = 0.025*(2.*np.random.uniform()-1.) + 0.3
    rnd_z = 0.85
    rnd_roll = 2.*np.pi*np.random.uniform()
    rnd_pitch = 2.*np.pi*np.random.uniform()
    rnd_ywa = 2.*np.pi*np.random.uniform()
    rnd_quaternion = tf.transformations.quaternion_from_euler(rnd_roll,rnd_pitch,rnd_ywa)
    spawn_gazebo_model(spawn_sdf, name, name, 
                       rnd_x, rnd_y, rnd_z,
                       rnd_quaternion[0],rnd_quaternion[1],rnd_quaternion[2],rnd_quaternion[3])
    rospy.sleep(0.01)
    
    j = 0
    np.random.seed(j)
    while j < num_trials:
        rospy.sleep(1.)
        
        pnp.move_to_start(open_gripper=True)
        observation = get_rgbd_image()
        color_data = observation["color"][cy-wy:cy+wy,cx-wx:cx+wx,:]
        depth_data = np.float32(observation["depth"][cy-wy:cy+wy,cx-wx:cx+wx])*0.001

        try:
            action = best_action(policy, depth_data, color_data)
        except NoValidGraspsException:
            j += 1
            np.random.seed(j)
            set_object_state(set_model,name,0.9)
            continue

        center_px, center_py = action.grasp.center.data
        center_py = int(center_py)
        center_px = int(center_px)
        center_depth = depth_data[center_py,center_px]
        gripper_angle = action.grasp.angle

        center_px = center_px - wx + cx
        center_py = center_py - wy + cy

        depth_camera_info = observation["depth_camera_info"]
        K = np.asarray(depth_camera_info.K).reshape([3,3])

        projected_eef_position = np.asarray([center_px,center_py,1.])
        scaled_eef_position = np.matmul(np.linalg.inv(K),projected_eef_position)
        scaled_eef_position = scaled_eef_position*center_depth
        scaled_eef_position[2] = action.grasp.depth
        eef_position = np.matmul(depth_cam_rotation_matrix[:3,:3],scaled_eef_position) + depth_cam_rotation_matrix[:3,3]

        approaching_axis = np.asarray([0.,0.,-1.])
        gripper_axis = np.asarray([-np.sin(gripper_angle),-np.cos(gripper_angle),0.])

        x_axis = np.cross(gripper_axis, approaching_axis) 
        y_axis = gripper_axis
        z_axis = approaching_axis

        gripper_rotation_mtx = np.identity(4)
        gripper_rotation_mtx[:3,0] = x_axis
        gripper_rotation_mtx[:3,1] = y_axis
        gripper_rotation_mtx[:3,2] = z_axis
        eef_quaternion = tf.transformations.quaternion_from_matrix(gripper_rotation_mtx)

        grasp_pose = geometry_msgs.msg.Pose()
        grasp_pose.position.x = eef_position[0]
        grasp_pose.position.y = eef_position[1]
        grasp_pose.position.z = eef_position[2] - 0.9115
        grasp_pose.orientation.x = eef_quaternion[0]
        grasp_pose.orientation.y = eef_quaternion[1]
        grasp_pose.orientation.z = eef_quaternion[2]
        grasp_pose.orientation.w = eef_quaternion[3]

        if eef_position[2] - 0.02 > 0.755:
            grasp_pose.position.z = 0.757 - 0.9115

        pnp.move_to_ready(open_gripper=True)
        pnp.pick(grasp_pose)
        pnp.move_to_ready(open_gripper=False)
        pnp.move_to_start(open_gripper=False)

        grasp_result = get_grasp_result(name)
        model_grasp_result_list[model_idx,j,0] = grasp_result
        model_grasp_result_list[model_idx,j,1] = action.q_value
        
        j+=1
        np.random.seed(j)
        set_object_state(set_model,name,0.85)
    set_object_state(set_model,name,-100.0)

Grasp Quality : 0.857274
[INFO] [1567189100.653276, 42.528000]: 12: Contact with Gripper
Grasp Quality : 0.863942
[INFO] [1567189123.342961, 64.612000]: 30: Contact with Gripper
Grasp Quality : 0.853775
[INFO] [1567189146.023358, 86.652000]: 39: Contact with Gripper
Grasp Quality : 0.838807
[INFO] [1567189169.419925, 109.403000]: 58: Contact with Gripper
Grasp Quality : 0.841355
[INFO] [1567189192.906699, 132.257000]: 78: Contact with Gripper
Grasp Quality : 0.859596
[INFO] [1567189216.040867, 154.828000]: 98: Contact with Gripper
Grasp Quality : 0.865225
[INFO] [1567189238.615802, 176.755000]: 117: Contact with Gripper
Grasp Quality : 0.868260
[INFO] [1567189260.195074, 197.757000]: 132: Contact with Gripper
Grasp Quality : 0.731980
[INFO] [1567189282.336749, 219.171000]: 148: Contact with Gripper
Grasp Quality : 0.743108
[INFO] [1567189304.788921, 240.844000]: 164: Contact with Gripper
Grasp Quality : 0.957613
[INFO] [1567189339.498097, 263.659000]: 9: No Contact with Gripper
Grasp Q

Grasp Quality : 0.002667
[INFO] [1567199715.588484, 2017.188000]: 9: Contact with Gripper
Grasp Quality : 0.001479
[INFO] [1567199978.335136, 2039.821000]: 14: No Contact with Gripper
Grasp Quality : 0.009458
[INFO] [1567200255.261503, 2062.239000]: 18: No Contact with Gripper
Grasp Quality : 0.014611
[INFO] [1567200528.075117, 2083.144000]: 24: No Contact with Gripper
Grasp Quality : 0.004253
[INFO] [1567200826.062675, 2105.324000]: 28: Contact with Gripper
Grasp Quality : 0.044365
[INFO] [1567201137.592480, 2128.721000]: 32: No Contact with Gripper
Grasp Quality : 0.002583
[INFO] [1567201418.368239, 2149.182000]: 37: No Contact with Gripper
Grasp Quality : 0.005591
[INFO] [1567201734.273342, 2172.021000]: 42: No Contact with Gripper
Grasp Quality : 0.004848
[INFO] [1567202060.128807, 2194.498000]: 47: No Contact with Gripper
Grasp Quality : 0.995848
[INFO] [1567202395.357482, 2217.439000]: 2: No Contact with Gripper
Grasp Quality : 0.005266
[INFO] [1567202714.094732, 2239.977000]: 7:

KeyboardInterrupt: 

In [15]:
with open("./GQCNN4.0-results.npz", 'wb') as f_res:
    np.savez(f_res, model_grasp_result_list)

In [18]:
for i in range(len(model_grasp_result_list)):
    print(i,np.mean(model_grasp_result_list[i],axis=0))

0 [1.         0.83233219]
1 [0.1        0.23329149]
2 [0.9       0.8535794]
3 [0.5        0.87507903]
4 [0.9        0.76034483]
5 [0.7        0.86052691]
6 [0.3        0.46000593]
7 [0.         0.07327962]
8 [0.9       0.8679819]
9 [0.2        0.10334062]
10 [0.2        0.18020207]
11 [0.7        0.87083471]
12 [0.2        0.11306132]
13 [0. 0.]
14 [0. 0.]
15 [0. 0.]
16 [0. 0.]
17 [0. 0.]
18 [0. 0.]
19 [0. 0.]
20 [0. 0.]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]